# Team 5 - HW5 - ETL Data Mart HW
Data Warehousing - Fall 21

Team 5: Vince Purcell, Brandon Mondile

In [55]:
import csv
from decimal import Decimal
from os import system
import random
import datetime
from datetime import date
import math
import sqlite3 as lite
from sqlite3 import Error

## Deliverable 2

Creation of Source Dimension Table and Updates to Product Dimension Table

In [56]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

def strip_uni(s):
      return s.replace(u'\xa0', u' ')

cur.execute("DROP TABLE IF EXISTS ProductDimension")
s = 'CREATE TABLE ProductDimension(ProductKey INT, SKU varchar(8), ProductName varchar(50), ProductClassID INT,'
s = s+' Subcategory varchar(50), Category varchar(50), Department varchar(50), ProductFamily varchar(50), Size varchar(10),' 
s = s+'PerCase INT, Manufacturer varchar(50), Supplier varchar(16))'
cur.execute(s)

cur.execute('drop table if exists SourceDimension')
cur.execute('create Table SourceDimension(SKU INT, Source INT)')

product_classes = {}
with open('product_class.csv', 'r') as csvfile:
    for row in csv.DictReader(csvfile):
        product_dict = {
            "product_class_id" : list(row.values())[0], 
            "product_subcategory" : row['product_subcategory'], 
            "product_category" : row['product_category'],
            "product_department" : row['product_department'], 
            "product_family" : row['product_family']
        }
        product_classes[row['product_subcategory']] = product_dict

i = 0
perCase = 12
with open('products_first_working.csv', 'r') as csvfile:
    for row in csv.DictReader(csvfile):
        i = i + 1
        sku = row['SKU']
        source = int(row['Source'])
        size = strip_uni(row['Size'])
        subcat = row['itemType']
        manufacturer = row['Manufacturer']
        productName = row['Product']

        product_class = product_classes[subcat]
        classId = int(product_class["product_class_id"])
        category = product_class["product_category"]
        department = product_class["product_department"]
        family = product_class["product_family"]

        supplier = "Rowan Warehouse"
        if subcat == "Milk":
            supplier = "Rowan Dairy"


        cur.execute("INSERT INTO SourceDimension VALUES (?,?)",
                        (sku, source))

        cur.execute("INSERT INTO ProductDimension VALUES (?,?,?,?,?,?,?,?,?,?,?,?)",
                        (i, sku, productName, classId, subcat, category, department, family, size, perCase, manufacturer, supplier))

        if ((i % 1000)==0):
            print("inserted row ",str(i))
            con.commit()
        


con.commit()
con.close()

inserted row  1000
inserted row  2000


## Deliverable 3

Store Dimension Table

In [57]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('drop table if exists StoreDimension')

s = 'CREATE TABLE StoreDimension('
s += 'StoreKey INT, StoreManager varchar(50), StoreStreetAddr varchar(50), StoreTown varchar(50),'
s += 'StoreZipCode varchar(15), StorePhone varchar(15), StoreState varchar(20))'
cur.execute(s)

key     = 5
manager = 'Vince Purcell'
addr    = '5 Main St.'
town    = 'Moorestown'
zcode   = '08057'
phone   = '8675309'
state   = 'NJ'

cur.execute("INSERT INTO StoreDimension VALUES (?,?,?,?,?,?,?)", 
    (key, manager, addr, town, zcode, phone, state))

con.commit()
con.close()

## Deliverable 4

Date Dimension Table

In [58]:
# Below are lookup dictionaries used during date dimension generation

# Searched '2020 Holidays' on Google and these were what was listed
holidays = {
    date(2020, 1, 1)    : 'New Years Day',
    date(2020, 1, 20)   : 'MLK Day',
    date(2020, 2, 17)   : 'Washingtons Birthday',
    date(2020, 4, 10)   : 'Good Friday',
    date(2020, 5, 25)   : 'Memorial Day',
    date(2020, 7, 3)    : 'Independence Day',
    date(2020, 9, 7)    : 'Labor Day',
    date(2020, 10, 12)  : 'Columbus Day',
    date(2020, 11, 3)   : 'Election Day',
    date(2020, 11, 11)  : 'Veterans Day',
    date(2020, 11, 26)  : 'Thanksgiving',
    date(2020, 12, 25)  : 'Christmas'
}

solstice_and_equinox = {
    date(2020, 3, 21)  : 'Spring',
    date(2020, 6, 21)  : 'Summer',
    date(2020, 9, 23)  : 'Fall',
    date(2020, 12, 22) : 'Winter',
    date(2021, 3, 21)  : 'Spring',
}

months = {
    1  : 'January',
    2  : 'February',
    3  : 'March',
    4  : 'April',
    5  : 'May',
    6  : 'June',
    7  : 'July',
    8  : 'August',
    9  : 'September',
    10 : 'October',
    11 : 'November',
    12 : 'December'
}

fiscal_year_end = date(2020,7,31)

quarters = {
    date(2020, 2, 1)  : 3,
    date(2020, 5, 1)  : 4,
    date(2020, 8, 1)  : 1,
    date(2020, 11, 1) : 2,
    date(2021, 2, 1)  : 3,
}

In [59]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('drop table if exists DateDimension')

s = 'CREATE TABLE DateDimension('
s += 'DateKey INT, Date DATETIME, DayNumberInMonth INT, DayNumberInYear INT, WeekNumberInYear INT, MonthNum INT, '
s += 'MonthTxt varchar(20), Quarter INT, Year INT, FiscalYear INT, isHoliday BOOLEAN, isWeekend BOOLEAN, Season varchar(10))'
cur.execute(s)

simulation_start_date = date(2019, 12, 31)
simulation_end_date = date(2020, 12, 31)
current_date = simulation_start_date
simulation_length = 365

key = 0
dayNum = 0
quarter = 2
quarter_index = 0
season = 'Winter'
season_index = 0

fiscalYear = 2019

for iday in range(0, simulation_length):
    if (( (iday+1) % 20)==0):
        print("Working on Day ", iday + 1)
    current_date += datetime.timedelta(1)
    key += 1
    dayNum += 1

    weekend = False
    if current_date.weekday() >= 5:
        weekend = True
    
    if current_date.month == "1" and current_date.day == "1":
        dayNum = 1

    if current_date > fiscal_year_end:
        fiscalYear = 2020
    
    if current_date == list(solstice_and_equinox.keys())[season_index]:
        season = list(solstice_and_equinox.values())[season_index]
        season_index += 1

    if current_date == list(quarters.keys())[quarter_index]:
        quarter = list(quarters.values())[quarter_index]
        quarter_index += 1

    weekNum = current_date.isocalendar()[1]
    monthNum = current_date.month
    monthText = months[monthNum]
    isHoliday = current_date in holidays.keys()

    cur.execute("INSERT INTO DateDimension VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", 
        (key, current_date, current_date.day, dayNum, weekNum, monthNum, monthText, quarter, current_date.year, fiscalYear, isHoliday, weekend, season))

con.commit()
con.close()

Working on Day  20
Working on Day  40
Working on Day  60
Working on Day  80
Working on Day  100
Working on Day  120
Working on Day  140
Working on Day  160
Working on Day  180
Working on Day  200
Working on Day  220
Working on Day  240
Working on Day  260
Working on Day  280
Working on Day  300
Working on Day  320
Working on Day  340
Working on Day  360


## Deliverable 6